### Test enrichment of treatment-induced hyper/hypo-methylation sites
[TEISER](https://github.com/goodarzilab/TEISER)

Here, I aim to identify the genes that are hyper or hypo methylated as genesets, and see if they have enriched accross all genes; the input table is list of genes with control vs. treated fold change of RNA expression, RNA stability and translational efficiency. 

Prepare inputs

In [18]:
%%bash
mkdir -p combined_analysis/

### hyper_methylation gene list
# subset by threshold - logFC >= 2
awk '$2>=2{print $1}' meRIP-seq/hl60_delta_mtyl.txt | \
awk '!seen[$1]++' > combined_analysis/hyper_mtyl.txt

### hypo_methylation gene list
# subset by threshold - logFC < 2
awk '$2<=-2 {print $1}' meRIP-seq/hl60_delta_mtyl.txt | \
awk '!seen[$1]++' > combined_analysis/hypo_mtyl.txt

In [19]:
%%bash

declare -a Genesets=('hyper_mtyl' 'hypo_mtyl')
declare -a Experiments=(
# Ribo-seq
'Ribo-seq/hl60_delta_te.txt'

## HL-60 RNA-seq 
# RNA experssion 
'RNA-seq/hl60-exp/6h_delta_exp.txt' 'RNA-seq/hl60-exp/72h_delta_exp.txt' 'RNA-seq/hl60-exp/120h_delta_exp.txt' 
# RNA stability  
'RNA-seq/hl60-stbl/120h_delta_stbl.txt'  'RNA-seq/hl60-stbl/6h_delta_stbl.txt'

## 5 other AML cell lines RNA-seq
# RNA experssion 
'RNA-seq/other-exp/kg1_delta_exp.txt' 'RNA-seq/other-exp/molm14_delta_exp.txt'
'RNA-seq/other-exp/ociaml2_delta_exp.txt' 'RNA-seq/other-exp/ociaml3_delta_exp.txt'
'RNA-seq/other-exp/thp1_delta_exp.txt'
# RNA stability  
'RNA-seq/other-stbl/kg1_delta_stbl.txt' 'RNA-seq/other-stbl/molm14_delta_stbl.txt' 
'RNA-seq/other-stbl/ociaml2_delta_stbl.txt' 'RNA-seq/other-stbl/ociaml3_delta_stbl.txt'
'RNA-seq/other-stbl/thp1_delta_stbl.txt'
)

for exp in "${Experiments[@]}"; do
    for geneset in "${Genesets[@]}"; do
    
        echo $exp $geneset
        base=`basename $exp`
        base=${base/.txt/}
        
        # get intersect 
        awk 'NR==FNR{A[$1];next}$1 in A' $exp combined_analysis/${geneset}.txt > combined_analysis/${geneset}_${base}.txt
        
        perl $TEISERDIR/run_mi_gene_list.pl \
            --expfile=$exp \
            --genefile=combined_analysis/${geneset}_${base}.txt \
            --exptype=continuous \
            --ebins=7 \
            --species=human \
            --doremovedups=0 \
            --doremoveextra=0 &> combined_analysis/${geneset}_${base}.log
        # remove results from previous run 
        rm -fr combined_analysis/${geneset}_${base}_GENESET
        
        rm combined_analysis/${geneset}_${base}.txt
        mv ${exp}_GENESET combined_analysis/${geneset}_${base}_GENESET
        
        echo 'done!'
        
    done 

done

Ribo-seq/hl60_delta_te.txt hyper_mtyl
done!
Ribo-seq/hl60_delta_te.txt hypo_mtyl
done!
RNA-seq/hl60-exp/6h_delta_exp.txt hyper_mtyl
done!
RNA-seq/hl60-exp/6h_delta_exp.txt hypo_mtyl
done!
RNA-seq/hl60-exp/72h_delta_exp.txt hyper_mtyl
done!
RNA-seq/hl60-exp/72h_delta_exp.txt hypo_mtyl
done!
RNA-seq/hl60-exp/120h_delta_exp.txt hyper_mtyl
done!
RNA-seq/hl60-exp/120h_delta_exp.txt hypo_mtyl
done!
RNA-seq/hl60-stbl/120h_delta_stbl.txt hyper_mtyl
done!
RNA-seq/hl60-stbl/120h_delta_stbl.txt hypo_mtyl
done!
RNA-seq/hl60-stbl/6h_delta_stbl.txt hyper_mtyl
done!
RNA-seq/hl60-stbl/6h_delta_stbl.txt hypo_mtyl
done!
RNA-seq/other-exp/kg1_delta_exp.txt hyper_mtyl
done!
RNA-seq/other-exp/kg1_delta_exp.txt hypo_mtyl
done!
RNA-seq/other-exp/molm14_delta_exp.txt hyper_mtyl
done!
RNA-seq/other-exp/molm14_delta_exp.txt hypo_mtyl
done!
RNA-seq/other-exp/ociaml2_delta_exp.txt hyper_mtyl
done!
RNA-seq/other-exp/ociaml2_delta_exp.txt hypo_mtyl
done!
RNA-seq/other-exp/ociaml3_delta_exp.txt hyper_mtyl
done!
RNA-

rm: cannot remove 'combined_analysis/hyper_mtyl_hl60_delta_te_GENESET': No such file or directory
rm: cannot remove 'combined_analysis/hypo_mtyl_hl60_delta_te_GENESET': No such file or directory
rm: cannot remove 'combined_analysis/hyper_mtyl_6h_delta_exp_GENESET': No such file or directory
rm: cannot remove 'combined_analysis/hypo_mtyl_6h_delta_exp_GENESET': No such file or directory
rm: cannot remove 'combined_analysis/hyper_mtyl_72h_delta_exp_GENESET': No such file or directory
rm: cannot remove 'combined_analysis/hypo_mtyl_72h_delta_exp_GENESET': No such file or directory
rm: cannot remove 'combined_analysis/hyper_mtyl_120h_delta_exp_GENESET': No such file or directory
rm: cannot remove 'combined_analysis/hypo_mtyl_120h_delta_exp_GENESET': No such file or directory
rm: cannot remove 'combined_analysis/hyper_mtyl_120h_delta_stbl_GENESET': No such file or directory
rm: cannot remove 'combined_analysis/hypo_mtyl_120h_delta_stbl_GENESET': No such file or directory
rm: cannot remove 'co

Let's redraw heatmaps using `--min=-3 --max=3` thresholds for those plots which have smaller range of signals:

In [20]:
%%bash 
declare -a Genesets=('hyper_mtyl' 'hypo_mtyl')
declare -a Experiments=(
'6h_delta_stbl' '120h_delta_stbl' 
'kg1_delta_stbl' 'ociaml2_delta_stbl' 'molm14_delta_stbl' 
'ociaml3_delta_stbl' 'thp1_delta_stbl'
'hl60_delta_te'
)
for exp in "${Experiments[@]}"; do
    for geneset in "${Genesets[@]}"; do
        echo $exp $geneset    
        perl $TEISERDIR/Scripts/teiser_draw_matrix.pl \
        --pvmatrixfile=combined_analysis/${geneset}_${exp}_GENESET/${exp}.txt.matrix \
        --summaryfile=combined_analysis/${geneset}_${exp}_GENESET/${exp}.txt.summary \
        --expfile=combined_analysis/${geneset}_${exp}_GENESET/${exp}.txt \
        --quantized=0 \
        --colmap=$TEISERDIR/Scripts/HEATMAPS/cmap_1.txt \
        --order=0 \
        --min=-3 --max=3 \
        --cluster=5 &>> combined_analysis/${geneset}_${exp}.log
        
        echo 'done!'
        
    done 
done

6h_delta_stbl hyper_mtyl
done!
6h_delta_stbl hypo_mtyl
done!
120h_delta_stbl hyper_mtyl
done!
120h_delta_stbl hypo_mtyl
done!
kg1_delta_stbl hyper_mtyl
done!
kg1_delta_stbl hypo_mtyl
done!
ociaml2_delta_stbl hyper_mtyl
done!
ociaml2_delta_stbl hypo_mtyl
done!
molm14_delta_stbl hyper_mtyl
done!
molm14_delta_stbl hypo_mtyl
done!
ociaml3_delta_stbl hyper_mtyl
done!
ociaml3_delta_stbl hypo_mtyl
done!
thp1_delta_stbl hyper_mtyl
done!
thp1_delta_stbl hypo_mtyl
done!
hl60_delta_te hyper_mtyl
done!
hl60_delta_te hypo_mtyl
done!


___
Make `png` figures:

In [16]:
mkdir -p combined_analysis/

In [21]:
%%bash 
for pdf in combined_analysis/*_GENESET/*.txt.summary.pdf; do 
    png=${pdf/.pdf/.png}
    di=`dirname $pdf`
    out=`basename $di`
    outpng=${out/_GENESET/.png}
    outpdf=${out/_GENESET/.pdf}
    
    bash /rumi/shams/abe/Workflows/my_scripts/pdf2png.sh $pdf
    cp $pdf combined_analysis/plots/$outpdf
    mv $png combined_analysis/plots/$outpng
done 

combined_analysis/hyper_mtyl_120h_delta_exp_GENESET/120h_delta_exp.txt.summary.pdf > combined_analysis/hyper_mtyl_120h_delta_exp_GENESET/120h_delta_exp.txt.summary.png
done!
combined_analysis/hyper_mtyl_120h_delta_stbl_GENESET/120h_delta_stbl.txt.summary.pdf > combined_analysis/hyper_mtyl_120h_delta_stbl_GENESET/120h_delta_stbl.txt.summary.png
done!
combined_analysis/hyper_mtyl_6h_delta_exp_GENESET/6h_delta_exp.txt.summary.pdf > combined_analysis/hyper_mtyl_6h_delta_exp_GENESET/6h_delta_exp.txt.summary.png
done!
combined_analysis/hyper_mtyl_6h_delta_stbl_GENESET/6h_delta_stbl.txt.summary.pdf > combined_analysis/hyper_mtyl_6h_delta_stbl_GENESET/6h_delta_stbl.txt.summary.png
done!
combined_analysis/hyper_mtyl_72h_delta_exp_GENESET/72h_delta_exp.txt.summary.pdf > combined_analysis/hyper_mtyl_72h_delta_exp_GENESET/72h_delta_exp.txt.summary.png
done!
combined_analysis/hyper_mtyl_hl60_delta_te_GENESET/hl60_delta_te.txt.summary.pdf > combined_analysis/hyper_mtyl_hl60_delta_te_GENESET/hl60_del

Write README.md draft:

In [22]:
%%bash 
touch combined_analysis/README.md
for f in combined_analysis/*.png; do 
    b=`basename $f`
    t=${b/.png/}
    echo '#### '$t >> combined_analysis/README.md
    echo -e "<img src=\""$f"\" title=\""$t"\" style=\"width:1000px\">\n" >> combined_analysis/README.md
done 